# Word Generation In $G=\mathbb{Z}_2*\mathbb{Z}$

### Write $G=\langle x|x^2=1\rangle*\langle y \rangle$, with generating set $S=\{x,y,y^{-1}\}$.

Our interest is to generate all the **excursions** on the Cayley Graph of $G$ wrt $S$. These are words that end up back at one (Eg. $xx$ or $yy^{-1}$ or $yxxy^{-1}$). To avoid redundancy, we can generate **simple excursions**, those that don't go back to 1 "too early".

Let $Z_{g,X}$ be the set of words ($\in S^*$) that evaluates to $g\in G$ and contains no proper prefix in $X\subseteq G$.
Let $Z_{g,X}^{(n)}$ be such words of length $n$. In most cases, we only need $X\subseteq S\cup\{1\}$.

Our Goal is to find $Z_{1,\emptyset}^{(n)}$ for some small $n$. Contenations of such words of varied $n$ gives another such word. To avoid redundency, We shall compute $Z_{1,\{1\}}^{(n)}$, and notice that 

$$Z_{1,\emptyset}^{(n)}=\left(\cup_{\lambda}[\times_{i}Z_{1,\{1\}}^{(\lambda_i)}]\right)\cup Z_{1,\{1\}}^{(n)}$$

where $\lambda=(\lambda_i)$ ranges over the $2^{n-1}-1$ nontrivial compositions of $n$. Furthermore, the $2^{n-1}$ sets involved in the union expression are pairwise disjoint.


In [109]:
# Let 0,1,2 denote x,y,y^-1 respectively. Use e for the identity of G and the empty word

alph = [0,1,2] # Edit ONLY this line if you want to use different symbols

alph = list(map(str,alph))
nxt_alph_char = {alph[i]:alph[(i+1)] for i in range(len(alph)-1)}
nxt_alph_char[alph[-1]] = None
nxt_alph_char

{'0': '1', '1': '2', '2': None}

In [110]:
def char_inv(c):
    '''
    Recall: c in [0,1,2]
    Returns the inverse element of c
    '''
    return alph[(3-alph.index(c))%3]
    


class Word_Z2Z(object):
    def __init__(self,w=[]):
        self.w = []
        self.wred = [] # reduced word of w
        for c in w:
            self.add_char(c)
    
    def add_char(self,c):
        w = self.w
        wred = self.wred
        w.append(c)
        if self.wred[-1:]==[char_inv(c)]:
            wred.pop()
        else:
            wred.append(c)
    
    def del_last(self):
        # assume w is nonempty
        w = self.w
        c = w[-1]
        self.add_char(char_inv(c))
        w.pop()
        w.pop()
        return c
        
    def last(self):
        return self.w[-1]
        
    def word(self):
        return ''.join(map(str, self.w))
    
    def reduced_word(self):
        return ''.join(map(str, self.wred))
    
    def __len__(self):
        return len(self.w)
    
    def reduced_length(self):
        return len(self.wred)
        
    def __str__(self):
        return self.word()
    
    def __iter__(self):
        return iter(self.w)
    
    def clone(self):
        # bit more efficient than calling Word_Z2Z(self)
        other = Word_Z2Z()
        other.w = list(self.w)
        other.wred = list(self.wred)
        return other
    
    def clear(self):
        self.w = []
        self.wred = []
    

In [111]:
# initialize a new word generator
wgen = Word_Z2Z()

In [112]:
str(wgen)=='' and len(wgen)==0

True

In [113]:
from collections import defaultdict
# start simple. generate all words up to a given max length
maxlen = 5
wlist_by_len = defaultdict(list)
wd_reduction = {}

# depth first approach
#nit = 8

start = 1

while start or len(wgen):
    start = 0
    #print(wgen)
    wlist_by_len[len(wgen)].append(str(wgen))
    wd_reduction[str(wgen)] = wgen.reduced_word()
    if len(wgen)<maxlen:
        wgen.add_char(alph[0])
    else:
        while len(wgen) and nxt_alph_char[wgen.last()] is None:
            wgen.del_last()
        if len(wgen):
            wgen.add_char(nxt_alph_char[wgen.del_last()])
#     nit -=1
#     if not nit:
#         break

In [114]:
            
print(wlist_by_len)
print(wd_reduction)
len(wgen)==0        

defaultdict(<class 'list'>, {0: [''], 1: ['0', '1', '2'], 2: ['00', '01', '02', '10', '11', '12', '20', '21', '22'], 3: ['000', '001', '002', '010', '011', '012', '020', '021', '022', '100', '101', '102', '110', '111', '112', '120', '121', '122', '200', '201', '202', '210', '211', '212', '220', '221', '222'], 4: ['0000', '0001', '0002', '0010', '0011', '0012', '0020', '0021', '0022', '0100', '0101', '0102', '0110', '0111', '0112', '0120', '0121', '0122', '0200', '0201', '0202', '0210', '0211', '0212', '0220', '0221', '0222', '1000', '1001', '1002', '1010', '1011', '1012', '1020', '1021', '1022', '1100', '1101', '1102', '1110', '1111', '1112', '1120', '1121', '1122', '1200', '1201', '1202', '1210', '1211', '1212', '1220', '1221', '1222', '2000', '2001', '2002', '2010', '2011', '2012', '2020', '2021', '2022', '2100', '2101', '2102', '2110', '2111', '2112', '2120', '2121', '2122', '2200', '2201', '2202', '2210', '2211', '2212', '2220', '2221', '2222'], 5: ['00000', '00001', '00002', '0001

True

In [118]:
def is_reduced(w):
    for c1,c2 in zip(w[:-1],w[1:]):
        if c2==char_inv(c1):
            return False
    return True

# all the reduction should indeed be reduced
assert(all(map(is_reduced, wd_reduction.values())))

In [121]:
def gen_simple_excursions_up_to_length(n):
    wgen.clear()
    start = 1
    se_by_length = defaultdict(list)
    while start or len(wgen):
        if wgen.reduced_length()==0:
            # everything cancels. reduced word is the empty word, so we have an excursion
            se_by_length[len(wgen)].append(str(wgen))
            # for only simple excursions, must SKIP all the superstrings (Those with the current word as a proper prefix)
            # Note, in this case, the if statement below is not satisfied, so else block is entered.
            ## Thus, wgen becomes the next string in lex order that is at most the current length.
        if (start or wgen.reduced_length()) and len(wgen)<n and 2*wgen.reduced_length()<=n:
            # we don't count the empty word as a violating substring for simpleness.
            # The start variable helps make this exception.
            # if the reduced length is too long, we cannot "walk back in time" to form an excursion.
            wgen.add_char(alph[0])
        else:
            while len(wgen) and nxt_alph_char[wgen.last()] is None:
                wgen.del_last()
            if len(wgen):
                wgen.add_char(nxt_alph_char[wgen.del_last()])
        start = 0
    return se_by_length

In [ ]:
maxlen = 14
se_by_length = gen_simple_excursions_up_to_length(maxlen)

In [154]:
for n, ws in sorted(se_by_length.items()):
    print (n, ws)

0 ['']
2 ['00', '12', '21']
4 ['0120', '0210', '1002', '1122', '2001', '2211']
6 ['010020', '011220', '012120', '012210', '020010', '021120', '021210', '022110', '100002', '100122', '101202', '102102', '110022', '111222', '112002', '112122', '200001', '200211', '201201', '202101', '220011', '221001', '221211', '222111']
8 ['01000020', '01001220', '01002120', '01002210', '01012020', '01021020', '01100220', '01112220', '01120020', '01121220', '01122120', '01122210', '01210020', '01211220', '01212120', '01212210', '01220010', '01221120', '01221210', '01222110', '02000010', '02001120', '02001210', '02002110', '02012010', '02021010', '02110020', '02111220', '02112120', '02112210', '02120010', '02121120', '02121210', '02122110', '02200110', '02210010', '02211120', '02211210', '02212110', '02221110', '10000002', '10000122', '10001202', '10002102', '10010022', '10011222', '10012002', '10012122', '10100202', '10112202', '10120002', '10120122', '10121202', '10122102', '10200102', '10210002', '10

In [152]:
%time gen_simple_excursions_up_to_length(16); None

CPU times: user 2min 35s, sys: 0 ns, total: 2min 35s
Wall time: 2min 35s
